In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')

In [ ]:
train_data.head()

In [ ]:
train_data.shape

In [ ]:
import numpy as np
import pandas as pd
import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# !pip install contractions
# import contractions
import re
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud

In [ ]:
def preprocess(sentence):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    negative = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except',
                        'even though', 'yet']
    stop_words = [z for z in stop_words if z not in negative]
    preprocessed_tokens = [lemmatizer.lemmatize(temp.lower()) for temp in sentence.split() if temp not in stop_words] #lemmatization
    sentence_temp = re.sub('[^A-z]', ' ', ' '.join([x for x in preprocessed_tokens]).strip())
    return sentence_temp

In [ ]:
 train_data['excerpt'] = train_data['excerpt'].apply(lambda x: preprocess(x))

In [ ]:
x = train_data['excerpt']
y = train_data['target']
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)

In [ ]:
# # !pip install transformers
# import transformers

In [ ]:
# from transformers import BertTokenizer, RobertaTokenizer
# tokenizer = BertTokenizer.from_pretrained('../input/huggingface-bert/bert-base-uncased', do_lower_case=True)

In [ ]:
# import numpy as np
# import tensorflow as tf 

In [ ]:
# def bert_encode(data,maximum_length) :
#     input_ids = []
#     attention_masks = []
#     token_type_ids = []
#     for i in data:
#         encoded = tokenizer.encode_plus(

#         i,
#         add_special_tokens=True,
#         max_length=maximum_length,
#         pad_to_max_length=True,

#         return_attention_mask=True,

#         )

#         input_ids.append(encoded['input_ids'])
#         attention_masks.append(encoded['attention_mask'])
# #         token_type_ids.append(encoded['token_type_ids'])
        
#     return np.array(input_ids),np.array(attention_masks)  #, np.array(token_type_ids)

In [ ]:
max_len = max([len(x) for x in train_data['excerpt']])
max_len


In [ ]:
max_len = 800

In [ ]:
# train_input_ids,train_attention_masks = bert_encode(xtrain,max_len)
# test_input_ids,test_attention_masks = bert_encode(xtest,max_len)

In [ ]:
# train_input_ids,train_attention_masks = bert_encode(train_data,max_len)
# test_input_ids,test_attention_masks = bert_encode(test,60)

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.optimizers import Adam
# from keras.metrics import RootMeanSquaredError
# def create_model(bert_model, max_len):
#     input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
#     attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
# #     token_ids = tf.keras.Input(shape=(max_len,),dtype='int32')

#     output = bert_model([input_ids, attention_masks])[0]
#     output = output[:, 0, :]
# #     output = tf.keras.layers.Dense(512,activation='relu')(output)
# #     output = tf.keras.layers.Dropout(0.2)(output)
    
#     output = tf.keras.layers.Dense(256,activation='relu')(output)
#     output = tf.keras.layers.Dropout(0.4)(output)
    
#     output = tf.keras.layers.Dense(64, activation='relu')(output)
#     output = tf.keras.layers.Dropout(0.2)(output)

#     output = tf.keras.layers.Dense(1,activation='linear')(output)
#     model = tf.keras.models.Model(inputs = [input_ids, attention_masks],outputs = output)
#     model.compile(Adam(lr=2e-6), loss='mean_squared_error', metrics=[RootMeanSquaredError()])
#     return model

In [ ]:
# from transformers import TFBertModel, TFRobertaModel
# bert_model = TFBertModel.from_pretrained('../input/huggingface-bert/bert-base-uncased')

In [ ]:
# model = create_model(bert_model, max_len)
# model.summary()

In [ ]:
# history = model.fit([train_input_ids, train_attention_masks], train_data.target, validation_split=0.2, epochs=4, batch_size=4)

In [ ]:
# result = model.predict([test_input_ids, test_attention_masks])

In [ ]:
# from sklearn.metrics import mean_squared_error
# mean_squared_error(ytest, result)

In [ ]:
%cd ../input/sentence-transformers/sentence-transformers-master

In [ ]:
import sentence_transformers
from sentence_transformers import SentenceTransformer
sent_model = SentenceTransformer('../../sentence-transformer-models/distilroberta-base-paraphrase-v1')

In [ ]:
xtrain_encode = sent_model.encode([temp for temp in xtrain])
xtest_encode = sent_model.encode([temp for temp in xtest])

In [ ]:
# xtrain_encode = sent_model.encode([temp for temp in x])
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(xtrain_encode, ytrain)
ypred = reg.predict(xtest_encode)

from sklearn.metrics import mean_squared_error
mean_squared_error(ytest, ypred)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV,Lasso,LassoCV
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics import mean_squared_error
import warnings

lr=0.1
n = 20
kf = KFold(n_splits=5, random_state=42, shuffle=True)
svr_model = SVR(C=1)
decision_tree_model = DecisionTreeRegressor()
random_forest_model = RandomForestRegressor(n_estimators=n)
adaboost_model = AdaBoostRegressor(n_estimators=n,learning_rate=lr)
gradientboost_model = GradientBoostingRegressor(learning_rate=lr,n_estimators = n)
ridge_model = Ridge(alpha=0.1)
ridge_cv_model = RidgeCV(alphas=[0.1,0.01,0.001],cv=5)
lasso_model = Lasso(alpha=0.1)
lasso_cv_model = LassoCV(alphas=[0.1,0.01,0.001],cv=5)

In [ ]:
def metrics(ytest,ypred):
    return np.sqrt(mean_squared_error(ytest,ypred))

In [ ]:
xtrain_encode.shape

In [ ]:
xtrain_encode = sent_model.encode([temp for temp in xtrain])
xtest_encode = sent_model.encode([temp for temp in xtest])

In [ ]:
result = pd.DataFrame([],columns=['Model','CV_rmse','Prediction_rmse'])
def compute(model,i):
#     cv_rmse = np.sqrt(-cross_val_score(model, xtrain_encode, ytrain, scoring="neg_mean_squared_error", cv=kf))
    model.fit(xtrain_encode,ytrain)
    ypred = model.predict(xtest_encode)
    result.loc[i] = [str(model)[:str(model).index('(')] ,4,metrics(ytest,ypred)]

In [ ]:
models = [svr_model,decision_tree_model,random_forest_model,adaboost_model,gradientboost_model,ridge_model,ridge_cv_model,lasso_model,lasso_cv_model]
for model in range(len(models)):
    print(model)
    compute(models[model],model)
result = result.sort_values('CV_rmse')
warnings.filterwarnings("ignore")
result

In [ ]:
X = sent_model.encode(x) 

In [ ]:
from mlxtend.regressor import StackingCVRegressor
stack_gen = StackingCVRegressor(regressors=(ridge_model,ridge_cv_model,lasso_cv_model),
                                meta_regressor=svr_model,
                                use_features_in_secondary=True,cv=20)
stack_gen.fit(np.array(X),np.array(y))

In [ ]:
ypred = stack_gen.predict(np.array(xtest_encode))
warnings.filterwarnings("ignore")
print('StackingCV Regressor RMSE {}'.format(metrics(ytest,ypred)))

In [ ]:
# reg.fit(xtrain_encode, y)

In [ ]:
%cd ../../../working

In [ ]:
test_data = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
test_data.head()

In [ ]:
test_data['excerpt'] = test_data['excerpt'].apply(lambda x: preprocess(x))
# test_input_ids, test_attention_masks = bert_encode(test_data, max_len)

In [ ]:
xtest_encode = sent_model.encode([x for x in test_data['excerpt']])
ypred = stack_gen.predict(xtest_encode)
# from sklearn.metrics import mean_squared_error
# error = mean_squared_error(ytest, ypred)
# error

In [ ]:
# result = model.predict([test_input_ids, test_attention_masks])

In [ ]:
submission_df  = pd.read_csv('/kaggle/input/commonlitreadabilityprize/sample_submission.csv')
submission_df.head()

In [ ]:
submission_df['target'] = ypred
submission_df

In [ ]:
submission_df.to_csv('submission.csv', index = False)